# This code contains all the process of extraction and processing of the images from the ACDC Dataset, and the calculation of the topological descriptors.

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import os
import nibabel as nib
from gudhi import CubicalComplex
from gudhi.wasserstein import wasserstein_distance
from gudhi.representations import Entropy
import itertools
from itertools import groupby
import json

## For this code to work properly, the ACDC Dataset should be downloaded without changes from https://humanheart-project.creatis.insa-lyon.fr/database/#collection/637218c173e9f0047faa00fb and there should be an nnunet folder with the model used for segmentation (see nnUNet_README). We provide trial data (just one patient and its nnUNET segmentation masks) that allows testing of this notebook. The next flag controls the use of either option (False means default use, True means Trial use)

In [24]:
trial_flag=True

### Information about the patients

In [25]:
#Extract the end-diastole and end-systole frame number for each patient.
if trial_flag:
    base_path="Trial/patient/"
    range_database=2
else:
    base_path="ACDC/ACDC/database/"
    range_database=151

duplets = []

# This loop is used to extract the data from the ACDC Dataset directly
for i in range(1,range_database):
    patient_path="patient"
    if (i <= 100):
        info_path=base_path+"training/"
        if (i < 100):
            patient_path=patient_path+"0"
            if (i < 10):
                patient_path=patient_path+"0"
    else:
        info_path=base_path+"testing/"
    
    filepath=info_path+patient_path+str(i)+"/"+"Info.cfg"

    with open(filepath, "r") as file:
        ed_value = None
        es_value = None
            
        for line in file:
            if line.startswith("ED:"):
                ed_value = int(line.split(":")[1].strip())
            if line.startswith("ES:"):
                es_value = int(line.split(":")[1].strip())
            
        if ed_value is not None and es_value is not None:
            duplets.append((ed_value, es_value))

### Extraction of 3D Images and ROUSEG segmentation

In [26]:
#Normalization between 0 and 1
def normalize (image):
    min_value = np.min(image)  
    max_value = np.max(image) 
        
    image = (image - min_value) / (max_value - min_value)
    
    return image

In [27]:
#This next cell does:
#1. Save the 3D frames that are in the ED-ES range for each patient's video, so that we can generate masks with the nnUNet model
#2. Apply the ROUSEG mask, normalize the three images and save them to calculate their persistence diagrams

saved_images_dir="3DImagesPreFilter"
if not os.path.exists(saved_images_dir):
    os.mkdir(saved_images_dir)
rouseg_filtered_dir="ROUSEG"
if not os.path.exists(rouseg_filtered_dir):
    os.mkdir(rouseg_filtered_dir)

for i in range(1,range_database):
    patient_path="patient"
    if (i <= 100):
        image_path=base_path+"training/"
        if (i < 100):
            patient_path=patient_path+"0"
            if (i < 10):
                patient_path=patient_path+"0"
    else:
        image_path=base_path+"testing/"
        
    file_path=image_path+patient_path+str(i)+"/"+patient_path+str(i)+"_4d.nii.gz" #Video

    #Take the ROUSEG Filter, which is the end-systole mask
    if duplets[i-1][1]<10:
        filter_path=image_path+patient_path+str(i)+"/"+patient_path+str(i)+"_frame0"+str(duplets[i-1][1])+"_gt.nii.gz"
    else:
        filter_path=image_path+patient_path+str(i)+"/"+patient_path+str(i)+"_frame"+str(duplets[i-1][1])+"_gt.nii.gz"

    img = nib.load(file_path)
    img_data = img.get_fdata()
    
    filter_img = nib.load(filter_path)
    filter_img = filter_img.get_fdata()

    rouseg_filtered_dir_R=rouseg_filtered_dir+"/Right"
    if not os.path.exists(rouseg_filtered_dir_R):
        os.mkdir(rouseg_filtered_dir_R)

    rouseg_filtered_dir_M=rouseg_filtered_dir+"/Myoca"
    if not os.path.exists(rouseg_filtered_dir_M):
        os.mkdir(rouseg_filtered_dir_M)

    rouseg_filtered_dir_L=rouseg_filtered_dir+"/Left"
    if not os.path.exists(rouseg_filtered_dir_L):
        os.mkdir(rouseg_filtered_dir_L)

    for t in range(duplets[i-1][0],duplets[i-1][1]+1):
        # Take the 3D slice corresponding to time t-1
        img_3d_data = img_data[:, :, :, t-1]
        img_3d = nib.Nifti1Image(img_3d_data, img.affine, img.header)

        # The saving format is the concatenation of the three-digit number of the patient and the two-digit number of the frame
        # The _0000 at the end is a condition for the nnUNet model and it makes no difference for us.
        output_file_path = os.path.join(saved_images_dir, (patient_path+str(i))[-3:]+f'{t:02d}_0000.nii.gz')
        nib.save(img_3d, output_file_path)

        unique_values = np.unique(filter_img)
        unique_values=unique_values[1:]
        filtered_images=[]
        for value in unique_values:
            mask = (filter_img == value)
            temp_image = img_3d_data * mask
            temp_image = normalize(temp_image)
            temp_image = nib.Nifti1Image(temp_image, img.affine, img.header)
            filtered_images.append(temp_image)

        #Save separated in Right, Myo i Left
        output_file_path = os.path.join(rouseg_filtered_dir_R, (patient_path+str(i))[-3:]+f'{t:02d}.nii.gz')
        nib.save(filtered_images[0], output_file_path)
        output_file_path = os.path.join(rouseg_filtered_dir_M, (patient_path+str(i))[-3:]+f'{t:02d}.nii.gz')
        nib.save(filtered_images[1], output_file_path)
        output_file_path = os.path.join(rouseg_filtered_dir_L, (patient_path+str(i))[-3:]+f'{t:02d}.nii.gz')
        nib.save(filtered_images[2], output_file_path)

### NETSEG Segmentation

In [28]:
#Filter the images with the nnUNet masks. We did not write any code for the generation of the nnUNet masks; it is performed directly on the console.
images_dir="3DImagesPreFilter/"
netseg_filtered_dir="NETSEG"
if not os.path.exists(netseg_filtered_dir):
    os.mkdir(netseg_filtered_dir)

if trial_flag:
    netseg_filters_dir="Trial/nnunet_masks/"
else:
    netseg_filters_dir="nnunet/output/"


#Only take the .nii.gz files
images_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".nii.gz")])
filters_files = sorted([f for f in os.listdir(netseg_filters_dir) if f.endswith(".nii.gz")])

#There should be exactly the same amount of images as filters
if len(images_files) != len(filters_files):
    raise ValueError("The number of .nii.gz files in the images and filters directories do not match.")

#Here we run through the image and filter directories simultaneously
for image_file, filter_file in zip(images_files, filters_files):
    image_path = os.path.join(images_dir, image_file)
    filter_path = os.path.join(netseg_filters_dir, filter_file)

    img = nib.load(image_path)
    img_data = img.get_fdata()

    filter_img = nib.load(filter_path)
    filter_img_data = filter_img.get_fdata()

    netseg_filtered_dir_R=netseg_filtered_dir+"/Right"
    if not os.path.exists(netseg_filtered_dir_R):
        os.mkdir(netseg_filtered_dir_R)

    netseg_filtered_dir_M=netseg_filtered_dir+"/Myoca"
    if not os.path.exists(netseg_filtered_dir_M):
        os.mkdir(netseg_filtered_dir_M)

    netseg_filtered_dir_L=netseg_filtered_dir+"/Left"
    if not os.path.exists(netseg_filtered_dir_L):
        os.mkdir(netseg_filtered_dir_L)

    # Take the different regions of the heart
    unique_values = np.unique(filter_img_data)
    unique_values=unique_values[1:] #Ignore the first value (background)
    filtered_images=[]
    for value in unique_values:
        mask = (filter_img_data == value)
        temp_image = img_data * mask
        temp_image = normalize(temp_image)
        temp_image = nib.Nifti1Image(temp_image, img.affine, img.header)
        filtered_images.append(temp_image)

    #Save the filtered images. The order is different than for ROUSEG because the nnUNet masks have different colors
    output_file_path = os.path.join(netseg_filtered_dir_L, filter_file)
    nib.save(filtered_images[0], output_file_path)
    output_file_path = os.path.join(netseg_filtered_dir_M, filter_file)
    nib.save(filtered_images[1], output_file_path)
    output_file_path = os.path.join(netseg_filtered_dir_R, filter_file)
    nib.save(filtered_images[2], output_file_path)

### Calculation and saving of topological descriptor lists

In [29]:
# The images start with the number of the patient: 001, 002,..., 150
def get_prefix(filename):
    return filename[:3]

#Function to extract the entropies and distance between diagrams.
def calculate_entropies_and_distance (directory): 

    dlist=[]
    e0list=[]
    e1list=[]
    e2list=[]

    files = sorted([f for f in os.listdir(directory) if f.endswith(".nii.gz")])
    
    for prefix, group in groupby(files, key=get_prefix):

        print(prefix)
        grouped_files = list(group)
        
        patient_distance_list=[]
        patient_e0_list=[]
        patient_e1_list=[]
        patient_e2_list=[]

        # We need to take both the current frame and the next one, to compute the distacnes
        for i, image_file in enumerate(grouped_files[:-1]):
            path1 = os.path.join(directory, image_file)
            path2 = os.path.join(directory, grouped_files[i+1])

        # In the first frame, we compute its diagram and the entropies
            if (image_file==grouped_files[0]):
    
                img_1 = nib.load(path1)
                img_data_1 = img_1.get_fdata()
        
                cubical_complex = CubicalComplex(top_dimensional_cells=img_data_1)
                cubical_complex.persistence()
                entropy = Entropy()
                
                diagram1_h0 = cubical_complex.persistence_intervals_in_dimension(0)
                if np.isinf(diagram1_h0[-1, 1]): #The infinite point is not taken into account in the calculation of entropy and distance so we remove it
                    diagram1_h0 = diagram1_h0[:-1]
                diagram1_h1 = cubical_complex.persistence_intervals_in_dimension(1)
                diagram1_h2 = cubical_complex.persistence_intervals_in_dimension(2)
        
                h0_entropy = entropy.fit_transform([diagram1_h0])[0][0]
                patient_e0_list.append(h0_entropy)
                
                h1_entropy = entropy.fit_transform([diagram1_h1])[0][0]
                patient_e1_list.append(h1_entropy)

                h2_entropy = entropy.fit_transform([diagram1_h2])[0][0]
                patient_e2_list.append(h2_entropy)

            # In the following frames, we have already calculated that diagram so no need to do it again
            else:
    
                diagram1_h0=diagram2_h0
                diagram1_h1=diagram2_h1
                diagram1_h2=diagram2_h2

            # In both cases, we compute the diagram corresponding to the next frame and compute the distance
            img_2 = nib.load(path2)
            img_data_2 = img_2.get_fdata()
    
            cubical_complex = CubicalComplex(top_dimensional_cells=img_data_2)
            cubical_complex.persistence()
            entropy = Entropy()
            
            diagram2_h0 = cubical_complex.persistence_intervals_in_dimension(0)
            if np.isinf(diagram2_h0[-1, 1]):
                    diagram2_h0 = diagram2_h0[:-1]
            diagram2_h1 = cubical_complex.persistence_intervals_in_dimension(1)
            diagram2_h2 = cubical_complex.persistence_intervals_in_dimension(2)
    
            h0_entropy = entropy.fit_transform([diagram2_h0])[0][0]
            patient_e0_list.append(h0_entropy)
            
            h1_entropy = entropy.fit_transform([diagram2_h1])[0][0]
            patient_e1_list.append(h1_entropy)

            h2_entropy = entropy.fit_transform([diagram2_h2])[0][0]
            patient_e2_list.append(h2_entropy)
    
            distance_h0 = wasserstein_distance(diagram1_h0, diagram2_h0, order=2)
            distance_h1 = wasserstein_distance(diagram1_h1, diagram2_h1, order=2)
            distance_h2 = wasserstein_distance(diagram1_h2, diagram2_h2, order=2)
    
            distance=distance_h0+distance_h1+distance_h2
    
            patient_distance_list.append(distance)
    
        dlist.append(patient_distance_list)
        e0list.append(patient_e0_list)
        e1list.append(patient_e1_list)
        e2list.append(patient_e2_list)

    return dlist, e0list, e1list, e2list

In [30]:
# Directory to save the generated entropy and distances
lists_dir="Lists/"
if not os.path.exists(lists_dir):
    os.mkdir(lists_dir)

In [32]:
def save_list(listt, list_name, directory):
    with open(directory+list_name,"w") as f:
        json.dump(listt, f)

In [33]:
R_dist_ROUSEG, R_e0_ROUSEG, R_e1_ROUSEG, R_e2_ROUSEG = calculate_entropies_and_distance ("ROUSEG/Right")
M_dist_ROUSEG, M_e0_ROUSEG, M_e1_ROUSEG, M_e2_ROUSEG = calculate_entropies_and_distance ("ROUSEG/Myoca")
L_dist_ROUSEG, L_e0_ROUSEG, L_e1_ROUSEG, L_e2_ROUSEG = calculate_entropies_and_distance ("ROUSEG/Left")
R_dist_NETSEG, R_e0_NETSEG, R_e1_NETSEG, R_e2_NETSEG = calculate_entropies_and_distance ("NETSEG/Right")
M_dist_NETSEG, M_e0_NETSEG, M_e1_NETSEG, M_e2_NETSEG = calculate_entropies_and_distance ("NETSEG/Myoca")
L_dist_NETSEG, L_e0_NETSEG, L_e1_NETSEG, L_e2_NETSEG = calculate_entropies_and_distance ("NETSEG/Left")

001
001
001
001
001
001


In [35]:
save_list(R_dist_ROUSEG, "R_dist_ROUSEG.json", lists_dir)
save_list(R_e0_ROUSEG, "R_e0_ROUSEG.json",lists_dir)
save_list(R_e1_ROUSEG, "R_e1_ROUSEG.json",lists_dir)
save_list(R_e2_ROUSEG, "R_e2_ROUSEG.json",lists_dir)
save_list(M_dist_ROUSEG, "M_dist_ROUSEG.json", lists_dir)
save_list(M_e0_ROUSEG, "M_e0_ROUSEG.json",lists_dir)
save_list(M_e1_ROUSEG, "M_e1_ROUSEG.json",lists_dir)
save_list(M_e2_ROUSEG, "M_e2_ROUSEG.json",lists_dir)
save_list(L_dist_ROUSEG, "L_dist_ROUSEG.json", lists_dir)
save_list(L_e0_ROUSEG, "L_e0_ROUSEG.json",lists_dir)
save_list(L_e1_ROUSEG, "L_e1_ROUSEG.json",lists_dir)
save_list(L_e2_ROUSEG, "L_e2_ROUSEG.json",lists_dir)

save_list(R_dist_NETSEG, "R_dist_NETSEG.json", lists_dir)
save_list(R_e0_NETSEG, "R_e0_NETSEG.json",lists_dir)
save_list(R_e1_NETSEG, "R_e1_NETSEG.json",lists_dir)
save_list(R_e2_NETSEG, "R_e2_NETSEG.json",lists_dir)
save_list(M_dist_NETSEG, "M_dist_NETSEG.json", lists_dir)
save_list(M_e0_NETSEG, "M_e0_NETSEG.json",lists_dir)
save_list(M_e1_NETSEG, "M_e1_NETSEG.json",lists_dir)
save_list(M_e2_NETSEG, "M_e2_NETSEG.json",lists_dir)
save_list(L_dist_NETSEG, "L_dist_NETSEG.json", lists_dir)
save_list(L_e0_NETSEG, "L_e0_NETSEG.json",lists_dir)
save_list(L_e1_NETSEG, "L_e1_NETSEG.json",lists_dir)
save_list(L_e2_NETSEG, "L_e2_NETSEG.json",lists_dir)